In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Loading dataset

In [ ]:
df=pd.read_excel("JICA Pattuvam Data full.xlsx")
df

In [ ]:
df[df['REMARKS']=='Maintenance Work at Intake']

In [ ]:
def understand_data(data) :
    
    return(pd.DataFrame({"Datatype":data.dtypes,
                         "No of NAs":data.isna().sum(),
                         "No of unique elements":data.nunique()}))
understand_data(df)

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'],errors='coerce')

In [ ]:
df['CLEAR WATER SUMP LEVEL IN M'].unique()

In [ ]:
df[df['CLEAR WATER SUMP LEVEL IN M']=='...']

In [ ]:
df[df['CLEAR WATER SUMP LEVEL IN M']=='......']

In [ ]:
df[df['CLEAR WATER SUMP LEVEL IN M']=='....']

In [ ]:
def convert_to_float(value):
    try:
        # Convert to float, handle different string formatting
        if isinstance(value,str):
            # Replace commas with dots for decimal places
            value = value.replace(',', '.')  
            value = value.replace(';', '.')
            value = value.replace('.,', '.')
            value = value.replace('..', '.')
            
            # Replace multiple dots with single dot
            value = value.replace('..', '.')  
            
            # Replace apostrophe with dot
            value = value.replace("'", ".")
            
            # Remove dot at the end if present
            if value.endswith('.'):
                value = value[:-1]
                
            if value.count('.') > 1:
                # Replace all dots except the first one
                value = value.replace('.', '', value.count('.') - 1)
            
   
            value = float(value)
            
        return value
    except (ValueError, TypeError):
        return np.nan  # Return NaN if conversion fails

df['CLEAR WATER SUMP LEVEL IN M'] = df['CLEAR WATER SUMP LEVEL IN M'].apply(convert_to_float)


df.isna().sum()


In [ ]:

float_values = []
for value in df['RAW WATER FLOW IN m3/h\t']:
    if isinstance(value, str):
        try:
            # Remove any spaces and other unexpected characters
            value = value.replace(' ', '.').replace('/', '.')
            # Convert to float
            float_value = float(value)
        except (ValueError, TypeError):
            print(f"Could not convert {value} to float")
            float_value = np.nan
    else:
        float_value = value  # keep the original float value

    float_values.append(float_value)

df['RAW WATER FLOW IN m3/h\t'] = float_values


df.isna().sum()

In [ ]:
# float_values = []
# for value in df['CLEAR WATER PUMPING FLOW m3/h']:
#     if isinstance(value, str):
#         try:
#             # Remove any spaces and other unexpected characters
#             value = value.replace(',', '.').replace('/', '.')
#             value = value.replace('..', '.').replace('/', '.')
#             value = value.replace(' ', '.').replace('/', '.')
#             # Remove dot at the end if present
#             if value.endswith('.'):
#                 value = value[:-1]
#             # Convert to float
#             float_value = float(value)
#         except (ValueError, TypeError):
#             print(f"Could not convert {value} to float")
#             float_value = np.nan
#     else:
#         float_value = value  # keep the original float value

#     float_values.append(float_value)

# df['CLEAR WATER PUMPING FLOW m3/h'] = float_values


# df.isna().sum()

In [ ]:
import numpy as np

float_values = []

for value in df['CLEAR WATER PUMPING FLOW m3/h']:
    if isinstance(value, str):
        try:
            # Remove any non-numeric characters except the first period
            value = ''.join(char if char.isdigit() or char == '.' else '' for char in value)
            
            # Handle cases where there are multiple periods (keep only the first one)
            if value.count('.') > 1:
                first_dot_index = value.find('.')
                value = value[:first_dot_index + 1] + value[first_dot_index + 1:].replace('.', '')
                
            # Remove trailing period
            if value.endswith('.'):
                value = value[:-1]
                
            # Convert to float
            float_value = float(value)
        except (ValueError, TypeError):
            print(f"Could not convert {value} to float")
            float_value = np.nan
    else:
        float_value = value  # keep the original float value

    float_values.append(float_value)

df['CLEAR WATER PUMPING FLOW m3/h'] = float_values

# Display the number of NaN values
df.isna().sum()


In [ ]:
df.dtypes

In [ ]:
df['DATE'].bfill(inplace=True)
df['TREATED WATER PRODUCTION IN m3/h'].fillna('NO DATA', inplace=True)
df['RAW WATER FLOW IN m3/h\t'].fillna('NO DATA', inplace=True)
df['CLEAR WATER SUMP LEVEL IN M'].fillna('NO DATA', inplace=True)
df['CLEAR WATER PUMPING FLOW m3/h'].fillna('NO DATA', inplace=True)
df['REMARKS'].fillna('No Remarks', inplace=True)

df.isnull().sum()

In [ ]:
df.dtypes

In [ ]:
df.rename(columns={'RAW WATER FLOW IN m3/h\t': 'RAW WATER FLOW IN m3/h'}, inplace=True)
flow_in_ML= ['RAW WATER FLOW IN m3/h','CLEAR WATER PUMPING FLOW m3/h',
       'TREATED WATER PRODUCTION IN m3/h','CLEAR WATER SUMP LEVEL IN M']
df[flow_in_ML] = df[flow_in_ML].apply(pd.to_numeric, errors='coerce')
df[flow_in_ML]= df[flow_in_ML]*0.001
df.rename(columns={'RAW WATER FLOW IN m3/h': 'RAW WATER FLOW IN ML','CLEAR WATER PUMPING FLOW m3/h':'CLEAR WATER PUMPING FLOW ML','TREATED WATER PRODUCTION IN m3/h':'TREATED WATER PRODUCTION IN ML'}, inplace=True)

In [ ]:
columns_of_interest = [
    'RAW WATER FLOW IN ML',
    'CLEAR WATER SUMP LEVEL IN M',
    'CLEAR WATER PUMPING FLOW ML',
    'TREATED WATER PRODUCTION IN ML'
]

for col in columns_of_interest:
    mean = df[col].mean()
    std_dev = df[col].std()
    
    # Calculate lower and upper bounds using Z-score method
    lower_limit = mean - 3 * std_dev  
    upper_limit = mean + 3 * std_dev 
    
    # Identify outliers
    outliers_before = df[(df[col] < lower_limit) | (df[col] > upper_limit)]
    
    # Print outliers before ffill
    print(f"Column: {col}")
    print(f"Lower Limit: {lower_limit}")
    print(f"Upper Limit: {upper_limit}")
    print('No. of outliers before ffill:', len(outliers_before))
    print(outliers_before)
    print("\n")
    
    # Forward fill outliers
    df.loc[outliers_before.index, col] = np.nan
    df[col] = df[col].ffill()
    
    # Identify outliers after ffill
    outliers_after = df[(df[col] < lower_limit) | (df[col] > upper_limit)]
    
    # Print outliers after ffill
    print('No. of outliers after ffill:', len(outliers_after))
    print(outliers_after)
    print("\n")
    


In [ ]:
df.dtypes

In [ ]:
df['DATE'] = pd.to_datetime(df['DATE'],errors='coerce')
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month 
df['MONTH-YEAR'] = df['DATE'].dt.strftime('%m-%Y')

In [ ]:
categories = {
    'Power Failure': ['Power Failure', 'Power failure', 'power failure', 'Power Failure at Intake', 'Power failure at intake', 'Power failure at Intake', 'power failed @9.45 to 2.25', 'power failed @9.45 to 2.26', 'power failed @9.45 to 2.27', 'power failed @9.45 to 2.28', 'power failed @9.45 to 2.29', 'power failed @9.45 to 2.30', 'Power failure @ intake', 'Permitted power failure.', 'power failure at 7.28 to 7.36 pm', 'Power failure at WTP', 'power failur', 'power failuire at intake', 'power failure at wtp 6.03 to6.40am', 'Power failure at intakeand WTP', 'voltage fluctation at intake', 
                      'Pumping stopped due to power failure at intake','under voltage'],
    'Maintenance': ['Maintenance Work at Intake', 'Pumping stopped due to maintenance at Thaliparamba', 'Pumping stopped due to maintenance work at Intake', 'distribution line maintanance', 'Gravity line air valve maintenance', 'Distribution line maintanance', 'stopped for maintanance', 'Pumping stopped due to pipe line maintanance at taliparamba', 'Pumping stopped for maintanance work & NHAI work.'],
    'Cleaning': ['intake chamber cleaning', 'Pumping stopped due to Intake chamber cleaning', 'Intake chamber cleaning', 'intake backwash', 'Pumping Stoped Due to permitted Power failure at intake', 'Pumping stopped due to  Intake chamber cleaning', 'channel cieaning', 'stopped due to intake backwash', 'intake back wash', 'inlet chamber cleaning'],
    'Others': ['Energy Auditing', 'Pumping Stoped due to intake Sump Level Low', 'Pumping Stopped due to intake Sump Level Low',  'RW pumping stopped', 'Raw Water Reading not available due to PLC complaint.'],
    'No Remarks':['No Remarks']}
def categorize_remark(remark):
    for category, keywords in categories.items():
        if any(keyword in str(remark) for keyword in keywords):
            return category
    return 'Others'

df['Remark Category'] = df['REMARKS'].apply(categorize_remark)

In [ ]:
df['Remark Category'].value_counts()

In [ ]:
df.dtypes

In [ ]:
df[df.duplicated()]

In [ ]:
df["TIME"].unique()

In [ ]:
df['TIME'] = df['TIME'].astype(str)

In [ ]:
def convert_time(time_str):
    parts = time_str.split()
    print(parts)
    print(len(parts))
    print(len(parts[0]))
    if len(parts) == 1:
        if len(parts[0]) == 1:
            return f'0{parts[0]}:00:00'
        elif len(parts[0]) == 2:
            print(len(parts[0]))
            return f'{parts[0]}:00:00'
    elif len(parts) == 2:
        return f'{parts[0]}:{parts[1]}:00'
    return time_str
df['TIME']= df['TIME'].apply(convert_time)

In [ ]:
df["TIME"].unique()

In [ ]:
df['DATETIME'] = pd.to_datetime(df['DATE'].astype(str) + ' ' + df['TIME'].astype(str))
df['HOUR'] = df['DATETIME'].dt.hour

In [ ]:
df

In [ ]:
df.to_excel('cleaned_pattuvam_data.xlsx',index=False)